In [1]:
# ===============================
# NAVER News Web Crawler _ df2(News Contents) Crawling  
# ===============================

# ---------------------
# * Process 
# ---------------------
# Step0: "Preparation" _ Import Packages 
# Step1: "Setting" 
#     1.1) Set File Directory & Parameters
#     1.2) Functions
# Step2: "Loop" - Find un-processed units & scrap the News 

# ---------------------
# * I/O 
# ---------------------
# Input : 
#     1. "naver_news_crawling_df1.csv" (df1): Contains [keywords, period, title, press, url]
# Output : 
#     1. "naver_news_crawling_df2.csv" (df2): Contains [title, date, contents] 


In [11]:
# ===============================
# Step0: "Preparation" _ Import Packages 
# ===============================
import requests
import time
from itertools import count
import datetime as dt
import time
import glob

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [12]:
# ================================
# Step1: "Setting" 
#     1.1) Set File Directory & Parameters
# ================================

# set file dir. 
team_dir = r'D:/02.users/yjin123/team3/' 
code_dir = r'D:/02.users/yjin123/team3/euro/euro_code/' 
data_dir = r'D:/02.users/yjin123/team3/euro/euro_data/' 
save_dir = r'D:/02.users/yjin123/team3/euro/euro_data/news_data/' 
logg_dir = r'D:/02.users/yjin123/team3/euro/euro_data/logger/' 

# set browser driver dir.
chrome_dir = team_dir+'chromedriver_win32 (1)/chromedriver'


# Today, now 
time_start = dt.datetime.now()

_today=time_start.date().strftime('%Y.%m.%d')
_time_start=time_start.time().strftime('%H:%M:%S')

# Set dir. for file saving
df1_name_head = "naver_news_crawling_df1"
df2_name_head = "naver_news_crawling_df2"

# Assign headless setting 
# headless_mode = True 
# if headless_mode :
#     options = webdriver.ChromeOptions()
#     options.add_argument('headless')    # add 'Headless' 
#     options.add_argument('window-size=1920x1080')    # set current display setting 
#     options.add_argument("disable-gpu") 
#     options.add_argument("lang=ko_KR") 

# Write Log 
verbose_ = False    # Print messages in console 

# Write log in logger.txt
logger_name = '1-02_logger_{0}_{1}.txt'.format(str(_today).replace('.',''), str(_time_start).replace(':',''))   
logg = open(logg_dir+logger_name,'w')

# Assign the End of Process datetime 
_EOP_time = '2020.10.26 08:30:00'
_EOP_ = dt.datetime.strptime(_EOP_time,'%Y.%m.%d %H:%M:%S')


In [13]:
# ================================
#     1.2) Functions
# ================================

# 1.2.1) 
def FindUnprocessedUnits(): 
    """
    뉴스데이터 저장 디렉토리에서 df1 목록과 df2 목록을 비교 후, 
    아직 df2 파일이 생성되지 못한 목록을 DataFrame으로 반환합니다.
    """
    # call every df1.csv file's directories & get unit names 
    df1_dirs = glob.glob(save_dir+df1_name_head+'*.csv')
    df1_ls = list(map(lambda x: x.split('\\'+df1_name_head)[1].replace('.csv',''), df1_dirs))
    
    # call every df2.csv file's directories & get unit names 
    df2_dirs = glob.glob(save_dir+df2_name_head+'*.csv')
#     df2_ls = list(map(lambda x: x.split('\\'+df2_name_head)[1].replace('.csv',''), df2_dirs))
#     df2_ls = list(map(lambda x: x.split('\\'+df2_name_head)[1].split('__')[0], df2_dirs))
    df2_ls = set(map(lambda x: x.split('\\'+df2_name_head)[1].split('__')[0], df2_dirs)) # 중복제거를 위해 set으로 
    
    # get total df1 dir.s & unit names as dataFrame 
    df1_df = pd.DataFrame(data={'df1_dirs': df1_dirs, 'files': df1_ls}) 
    
    # return un-crawled units DataFrame 
    return df1_df.loc[~df1_df['files'].isin(df2_ls), ].reset_index(drop=True)
    

In [14]:
# 1.2.2) 
# 함수 블록 ====================================
# parameter ------
# threshold_ = 1000

""" 함수개선사항 : 
1. 각 URL마다 들어가서 기사 수집 중, '예상치 못한 화면'(기사 호출이 안되서 크롤링에 실패하는 경우)
해당 df1_seg에 대해 '에러 띄우고', '중단'하고, 나머지 df로 계속 수집 수행하기 위한 
try except 추가 

2. 'df1'에 기사 내용이 하나도 없을때, '빈 df2'를 저장해주는 기능 추가 

3. log 기록 삽입 추가 
"""

def DF2Crawler(target_df, threshold_ = 1000):
    """ ====================function from ==========================="""
    # 0. Preparation
    # 개선사항2. 
    empty_df2 = pd.DataFrame(columns=['title','date','content','url'])
    
    for idx, val in target_df.iterrows(): # for 1: '개별df1파일 하나씩 루프 수행'
    #     keyword_ = val[1].split('_')[1]     # ex: 'BGF리테일'
    #     from_ = val[1].split('_')[2].replace('from')     # ex: '2010.01.01'
    #     to_ = val[1].split('_')[3].replace('to')     # ex: '2010.12.31'
    #     print(" * Keyword: {0}, From: {1}, To: {2}") 
    
        artcls_cnt = target_df.shape[0]
        dir_ = val[0] # 'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼...	'
        unit_ = val[1] # '_신세계인터내셔날_from2010.01.01_to2010.12.31'

        repo_ = " * Unit: '{0}' ({1}/{2}) ".format(unit_, idx+1, artcls_cnt)
        print(repo_)
        logg.write(repo_+'\n') 
        
        """ 이 안에 한 단위의 크롤러 삽입 """
        """ ====================indentx1 from ==========================="""
        # 단위(unit) 하나 전체 파일 호출 
        df1_whole = pd.read_csv(dir_) # 'Z:/0_프로젝트/데이터.._신세계인터내셔날_to2010.12.31.csv'호출

        # *** length가 0인 경우(기사가 없는 경우), 'continue': 다음 루프 수행
        # 기사가 있다면, 'pass' : 아래 코드 계속 수행 
        # if df1_whole.empty: continue
        if df1_whole.empty: 
            repo_ = '    No articles to scrap for "{0}"'.format(unit_) # '기사없음' 표시
            print(repo_) 
            logg.write(repo_+'\n') 
            save_filename = save_dir+df2_name_head+unit_+'__x.csv' 
            empty_df2.to_csv(save_filename, index=False) # 개선사항2: 빈 df2 저장 
            continue # for 1: 에 다음 루프로 

        # 전체 기사 수 선언
        tot_artc = df1_whole.shape[0]
        print('    The Number of articles:    {0} '.format(tot_artc))

        # threshold_ = 1000 # <<< parameter
        for n, i in enumerate(range(0, tot_artc, threshold_)): # for 2: '기사 1,000개단위로 루프 수행'
            start_ = i 
            end_ = i+threshold_ if i+threshold_<tot_artc else tot_artc
#             print(n, start_, end_)
            """ 1000줄이 넘는 df에 대해서, 해당 .iloc[,] 호출해서 데이터프레임 하나씩 수행 """
            """ ====================indentx2 from ==========================="""
            df1 = df1_whole.iloc[start_:end_,]

            dates=[]
            contents=[]
            df2_urls=[]

            # open blank browser
            driver2=webdriver.Chrome(chrome_dir)

            urls=df1['url'].tolist()
            titles=df1['title'].tolist()
            
            try: # : 개선사항1 try 1: 개별 URL호출 중 끊어진 경우, 다음 df1_seg 루프로 continue
                for url in urls:  # for 3: '개별 기사url 하나씩 루프 수행'
                    # 'urls' from df1: take 'each url' and do crawling
                    try:
                        driver2.get(url)
                        html=driver2.page_source
                        date=driver2.find_element_by_class_name('t11').text
                        #summary=driver.find_element_by_class_name('media_end_summary').text
                        content=driver2.find_element_by_class_name('_article_body_contents').text

                        dates.append(date)
                        contents.append(content)
                        df2_urls.append(url)
                    except: # when 'url' is None, (df1 has no 'None' values from '1-01_v.20.py' )
                        dates.append('NaN')
                        contents.append('NaN')
                        df2_urls.append('NaN')
            except: # : 개선사항1 except 1: 
                repo_ = '     >>> Failed to get News when URL: {0}'.format(url)
                print(repo_) 
                logg.write(repo_+'\n') 
                driver2.close() # 처리되지 못한 웹드라이버 close
                continue # for 2: 에 다음 루프로 

            # Export News Contenst DataFrame For the keyword (For whole period)
            # 데이터 프레임 생성 - 2. 제목, 날짜, 기사 본문
            try : # try 2: 
                mCols2=[]
                df_2=pd.DataFrame(mCols2)

                df_2['title']=titles # 'titles' from df1
                df_2['date']=dates
                df_2['content']=contents
                df_2['url']=df2_urls

                # export news list DataFrame to csv
                save_filename = save_dir+df2_name_head+unit_+'__{0}.csv'.format(n)

#             try : # try 2: <<< 위로 이동
                df_2.to_csv(save_filename, index=False)
                table_report = """    [{0}]_ from'{1}' to '{2}' (df2): successfully Saved  """.format(unit_, start_, end_-1)
                print(table_report)
                logg.write(table_report+'\n') 
            except Exception as e : # except 2: 
                table_report = """    >>> [{0}]_ from'{1}' to '{2}' (df2): failed to export .csv  """.format(unit_, start_, end_-1)
                print(table_report)
                logg.write(table_report+'\n') 

            driver2.close()

            """ ====================indentx2 to ==========================="""


        """ ====================indentx1 to ==========================="""
    """ ====================function to ==========================="""    

In [ ]:
# 3rd run

# ================================
# Step2: "Loop" - Find un-processed units & scrap the News 
# ================================

i_while = 1

while True : 
    # proc1. 미완결 파일 찾기 
    target_df = FindUnprocessedUnits()
    
#         # On-site ===========================================================
#     if target_df.empty: # 모든 파일에 대응하는 df2가 있는 경우 
#         print('\n *** End of Procedures : with {0} times of iteration'.format(i_while-1)) 
#         break # break the loop         
#     elif i_while > 100 : # 루프가 100번 돈 경우
#         break # break the loop         
#     else : # 진행되지 못한 df2가 아직 있는 경우 
#         print('\n  ========= {0} times of iteration: {1} files left to go'.format(i_while, target_df.shape[0]))
        

        # off-site ==========================================================
    if dt.datetime.now() > _EOP_ : # 지정 작업 종료 일시를 지난 경우, 종료 
        print('\n *** End of Procedures : with {0} times of iteration : EoP at {1}'.format(i_while-1, str(dt.datetime.now()))) 
        break # break the loop 
    elif target_df.empty: # 모든 파일에 대응하는 df2가 있는 경우 
        print('\n *** Temporary 1hrs pause : at {0} since there has no valid df1 files'.format(str(dt.datetime.now()))) 
        time.sleep(3600) # 1시간 일시정지 (df1이 생성될 시간을 확보) 
        continue # <<< 1시간 이상 걸리는 df1수집으로 target_df가 생성되지 않았을 경우에 detour 
    else : # 진행되지 못한 df2가 아직 있는 경우 
        print('\n  ========= {0} times of iteration: {1} files left to go'.format(i_while, target_df.shape[0]))
    
    # proc2. 크롤링 수행 
    try: 
        DF2Crawler(target_df, threshold_ = 1000) 
    except: 
        print('\n *** UnexpectedException ocurred: *************************** ')
              
    i_while += 1 # Number of iteration 

# save the log.txt 
logg.close() 


  ========= 1 times of iteration: 2179 files left to go
 * Unit: '_대웅_from2015.01.01_to2015.12.31' (1/2179) 
    The Number of articles:    1161 
    [_대웅_from2015.01.01_to2015.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_대웅_from2015.01.01_to2015.12.31]_ from'1000' to '1160' (df2): successfully Saved  
 * Unit: '_대웅제약_from2014.01.01_to2014.12.31' (2/2179) 
    The Number of articles:    1467 
    [_대웅제약_from2014.01.01_to2014.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_대웅제약_from2014.01.01_to2014.12.31]_ from'1000' to '1466' (df2): successfully Saved  
 * Unit: '_대원강업_from2014.01.01_to2014.12.31' (3/2179) 
    The Number of articles:    103 
    [_대원강업_from2014.01.01_to2014.12.31]_ from'0' to '102' (df2): successfully Saved  
 * Unit: '_대원미디어_from2014.01.01_to2014.12.31' (4/2179) 
    The Number of articles:    527 
    [_대원미디어_from2014.01.01_to2014.12.31]_ from'0' to '526' (df2): successfully Saved  
 * Unit: '_대원산업_from2014.01.01_to2014.12.31' (5/2179) 
  

 * Unit: '_대호_from2014.01.01_to2014.12.31' (44/2179) 
    The Number of articles:    1381 
    [_대호_from2014.01.01_to2014.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_대호_from2014.01.01_to2014.12.31]_ from'1000' to '1380' (df2): successfully Saved  
 * Unit: '_대호에이엘_from2014.01.01_to2014.12.31' (45/2179) 
    The Number of articles:    259 
    [_대호에이엘_from2014.01.01_to2014.12.31]_ from'0' to '258' (df2): successfully Saved  
 * Unit: '_대호피앤씨_from2014.01.01_to2014.12.31' (46/2179) 
    The Number of articles:    79 
    [_대호피앤씨_from2014.01.01_to2014.12.31]_ from'0' to '78' (df2): successfully Saved  
 * Unit: '_대화제약_from2014.01.01_to2014.12.31' (47/2179) 
    The Number of articles:    314 
    [_대화제약_from2014.01.01_to2014.12.31]_ from'0' to '313' (df2): successfully Saved  
 * Unit: '_대흥멀티통신_from2014.01.01_to2014.12.31' (48/2179) 
    No articles to scrap for "_대흥멀티통신_from2014.01.01_to2014.12.31"
 * Unit: '_더블유게임즈_from2014.01.01_to2014.12.31' (49/2179) 
    The Number of 

    [_동부_from2015.01.01_to2015.12.31]_ from'1000' to '1999' (df2): successfully Saved  
    [_동부_from2015.01.01_to2015.12.31]_ from'2000' to '2391' (df2): successfully Saved  
 * Unit: '_동부건설_from2014.01.01_to2014.12.31' (84/2179) 
    The Number of articles:    2091 
    [_동부건설_from2014.01.01_to2014.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_동부건설_from2014.01.01_to2014.12.31]_ from'1000' to '1999' (df2): successfully Saved  
    [_동부건설_from2014.01.01_to2014.12.31]_ from'2000' to '2090' (df2): successfully Saved  
 * Unit: '_동부라이텍_from2014.01.01_to2014.12.31' (85/2179) 
    The Number of articles:    283 
    [_동부라이텍_from2014.01.01_to2014.12.31]_ from'0' to '282' (df2): successfully Saved  
 * Unit: '_동부제철_from2014.01.01_to2014.12.31' (86/2179) 
    The Number of articles:    2143 
    [_동부제철_from2014.01.01_to2014.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_동부제철_from2014.01.01_to2014.12.31]_ from'1000' to '1999' (df2): successfully Saved  
    [_동부제철_from2

 * Unit: '_동양철관_from2014.01.01_to2014.12.31' (121/2179) 
    The Number of articles:    408 
    [_동양철관_from2014.01.01_to2014.12.31]_ from'0' to '407' (df2): successfully Saved  
 * Unit: '_동양철관_from2015.01.01_to2015.12.31' (122/2179) 
    The Number of articles:    454 
    [_동양철관_from2015.01.01_to2015.12.31]_ from'0' to '453' (df2): successfully Saved  
 * Unit: '_동양텔레콤_from2014.01.01_to2014.12.31' (123/2179) 
    The Number of articles:    41 
    [_동양텔레콤_from2014.01.01_to2014.12.31]_ from'0' to '40' (df2): successfully Saved  
 * Unit: '_동양파일_from2014.01.01_to2014.12.31' (124/2179) 
    The Number of articles:    120 
    [_동양파일_from2014.01.01_to2014.12.31]_ from'0' to '119' (df2): successfully Saved  
 * Unit: '_동양피엔에프_from2014.01.01_to2014.12.31' (125/2179) 
    The Number of articles:    66 
    [_동양피엔에프_from2014.01.01_to2014.12.31]_ from'0' to '65' (df2): successfully Saved  
 * Unit: '_동우팜투테이블_from2014.01.01_to2014.12.31' (126/2179) 
    No articles to scrap for "_동우팜투테이블_from

    [_디비케이_from2014.01.01_to2014.12.31]_ from'0' to '91' (df2): successfully Saved  
 * Unit: '_디스플레이텍_from2014.01.01_to2014.12.31' (161/2179) 
    The Number of articles:    69 
    [_디스플레이텍_from2014.01.01_to2014.12.31]_ from'0' to '68' (df2): successfully Saved  
 * Unit: '_디씨엠_from2014.01.01_to2014.12.31' (162/2179) 
    The Number of articles:    41 
    [_디씨엠_from2014.01.01_to2014.12.31]_ from'0' to '40' (df2): successfully Saved  
 * Unit: '_디아이_from2014.01.01_to2014.12.31' (163/2179) 
    The Number of articles:    1326 
    [_디아이_from2014.01.01_to2014.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_디아이_from2014.01.01_to2014.12.31]_ from'1000' to '1325' (df2): successfully Saved  
 * Unit: '_디아이_from2015.01.01_to2015.12.31' (164/2179) 
    The Number of articles:    738 
    [_디아이_from2015.01.01_to2015.12.31]_ from'0' to '737' (df2): successfully Saved  
 * Unit: '_디아이디_from2014.01.01_to2014.12.31' (165/2179) 
    The Number of articles:    283 
    [_디아이디_from2014.01

...

    [_아세아_from2015.01.01_to2015.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_아세아_from2015.01.01_to2015.12.31]_ from'1000' to '1255' (df2): successfully Saved  
 * Unit: '_아세아시멘트_from2014.01.01_to2014.12.31' (764/2179) 
    The Number of articles:    345 
    [_아세아시멘트_from2014.01.01_to2014.12.31]_ from'0' to '344' (df2): successfully Saved  
 * Unit: '_아세아제지_from2014.01.01_to2014.12.31' (765/2179) 
    The Number of articles:    114 
    [_아세아제지_from2014.01.01_to2014.12.31]_ from'0' to '113' (df2): successfully Saved  
 * Unit: '_아세아제지_from2015.01.01_to2015.12.31' (766/2179) 
    The Number of articles:    82 
    [_아세아제지_from2015.01.01_to2015.12.31]_ from'0' to '81' (df2): successfully Saved  
 * Unit: '_아세아텍_from2014.01.01_to2014.12.31' (767/2179) 
    The Number of articles:    170 
    [_아세아텍_from2014.01.01_to2014.12.31]_ from'0' to '169' (df2): successfully Saved  
 * Unit: '_아스타_from2014.01.01_to2014.12.31' (768/2179) 
    The Number of articles:    412 
    [_아스타_fr

 * Unit: '_아이텍스필_from2014.01.01_to2014.12.31' (806/2179) 
    No articles to scrap for "_아이텍스필_from2014.01.01_to2014.12.31"
 * Unit: '_아이티센_from2014.01.01_to2014.12.31' (807/2179) 
    The Number of articles:    308 
    [_아이티센_from2014.01.01_to2014.12.31]_ from'0' to '307' (df2): successfully Saved  
 * Unit: '_아이팩토리_from2014.01.01_to2014.12.31' (808/2179) 
    No articles to scrap for "_아이팩토리_from2014.01.01_to2014.12.31"
 * Unit: '_아인스M&M_from2014.01.01_to2014.12.31' (809/2179) 
    The Number of articles:    1 
    [_아인스M&M_from2014.01.01_to2014.12.31]_ from'0' to '0' (df2): successfully Saved  
 * Unit: '_아즈텍WB_from2014.01.01_to2014.12.31' (810/2179) 
    The Number of articles:    60 
    [_아즈텍WB_from2014.01.01_to2014.12.31]_ from'0' to '59' (df2): successfully Saved  
 * Unit: '_아진산업_from2014.01.01_to2014.12.31' (811/2179) 
    The Number of articles:    52 
    [_아진산업_from2014.01.01_to2014.12.31]_ from'0' to '51' (df2): successfully Saved  
 * Unit: '_아진엑스텍_from2014.01.01_to2014

    [_에너토크_from2014.01.01_to2014.12.31]_ from'0' to '59' (df2): successfully Saved  
 * Unit: '_에넥스_from2014.01.01_to2014.12.31' (854/2179) 
    The Number of articles:    1164 
    [_에넥스_from2014.01.01_to2014.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_에넥스_from2014.01.01_to2014.12.31]_ from'1000' to '1163' (df2): successfully Saved  
 * Unit: '_에듀아크_from2014.01.01_to2014.12.31' (855/2179) 
    The Number of articles:    2 
    [_에듀아크_from2014.01.01_to2014.12.31]_ from'0' to '1' (df2): successfully Saved  
 * Unit: '_에듀언스_from2014.01.01_to2014.12.31' (856/2179) 
    The Number of articles:    2 
    [_에듀언스_from2014.01.01_to2014.12.31]_ from'0' to '1' (df2): successfully Saved  
 * Unit: '_에듀패스_from2014.01.01_to2014.12.31' (857/2179) 
    The Number of articles:    2 
    [_에듀패스_from2014.01.01_to2014.12.31]_ from'0' to '1' (df2): successfully Saved  
 * Unit: '_에머슨퍼시픽_from2014.01.01_to2014.12.31' (858/2179) 
    The Number of articles:    676 
    [_에머슨퍼시픽_from2014.01.01_

    [_에스피지_from2014.01.01_to2014.12.31]_ from'0' to '303' (df2): successfully Saved  
 * Unit: '_에스피컴텍_from2014.01.01_to2014.12.31' (899/2179) 
    No articles to scrap for "_에스피컴텍_from2014.01.01_to2014.12.31"
 * Unit: '_에스피코프_from2014.01.01_to2014.12.31' (900/2179) 
    No articles to scrap for "_에스피코프_from2014.01.01_to2014.12.31"
 * Unit: '_에쎈테크_from2014.01.01_to2014.12.31' (901/2179) 
    The Number of articles:    56 
    [_에쎈테크_from2014.01.01_to2014.12.31]_ from'0' to '55' (df2): successfully Saved  
 * Unit: '_에쓰씨엔지니어링_from2014.01.01_to2014.12.31' (902/2179) 
    The Number of articles:    59 
    [_에쓰씨엔지니어링_from2014.01.01_to2014.12.31]_ from'0' to '58' (df2): successfully Saved  
 * Unit: '_에이디칩스_from2014.01.01_to2014.12.31' (903/2179) 
    The Number of articles:    47 
    [_에이디칩스_from2014.01.01_to2014.12.31]_ from'0' to '46' (df2): successfully Saved  
 * Unit: '_에이디테크놀로지_from2014.01.01_to2014.12.31' (904/2179) 
    The Number of articles:    96 
    [_에이디테크놀로지_from2014.01.01

 * Unit: '_엑사이엔씨_from2014.01.01_to2014.12.31' (948/2179) 
    The Number of articles:    133 
    [_엑사이엔씨_from2014.01.01_to2014.12.31]_ from'0' to '132' (df2): successfully Saved  
 * Unit: '_엑세스바이오_from2014.01.01_to2014.12.31' (949/2179) 
    The Number of articles:    576 
    [_엑세스바이오_from2014.01.01_to2014.12.31]_ from'0' to '575' (df2): successfully Saved  
 * Unit: '_엑스로드_from2014.01.01_to2014.12.31' (950/2179) 
    The Number of articles:    3 
    [_엑스로드_from2014.01.01_to2014.12.31]_ from'0' to '2' (df2): successfully Saved  
 * Unit: '_엑스씨이_from2014.01.01_to2014.12.31' (951/2179) 
    No articles to scrap for "_엑스씨이_from2014.01.01_to2014.12.31"
 * Unit: '_엑시콘_from2014.01.01_to2014.12.31' (952/2179) 
    The Number of articles:    16 
    [_엑시콘_from2014.01.01_to2014.12.31]_ from'0' to '15' (df2): successfully Saved  
 * Unit: '_엔브이에이치코리아_from2014.01.01_to2014.12.31' (953/2179) 
    The Number of articles:    119 
    [_엔브이에이치코리아_from2014.01.01_to2014.12.31]_ from'0' to '118' (df

In [8]:
""" 
동원_ : ' ... 용역동원', '강동원..' 등 같이 수집됨 
미래산업_ : 
서연_ : 
덕성_ : 
""" 

"\n동원_ : ' ... 용역동원', '강동원..' 등 같이 수집됨 \n미래산업_ : \n서연_ :  \n덕성_ : \n\n"

In [1]:
import sys
sys.pwd

AttributeError: module 'sys' has no attribute 'pwd'